In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import io
from PIL import Image
import os
import hashlib

In [ ]:
print(os.getcwd())

C:\Users\s_cam\src\Computer_Vision_Project\01_src\notebook


In [ ]:
DRIVER_PATH = './ext/chromedriver.exe'
output_path = './data/raw_google_images'
query = "star vogel"
count = 200

In [ ]:
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

C:\Users\s_cam\AppData\Local\Temp\ipykernel_42776\2505007267.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


In [ ]:
wd.get('https://google.com')

In [ ]:
#Cookies: click "Alle ablehnen" Button
button = wd.find_element(By.ID, 'W0wltc')
button.click()

In [ ]:
#search_box = wd.find_element(By.CSS_SELECTOR, 'input.gLFyf')
#search_box.send_keys('Dogs')

In [ ]:
#wd.quit

In [ ]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements(By.CSS_SELECTOR, 'img.Q4LuWd')
        print(type(thumbnail_results))
        print(thumbnail_results)
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements(By.CSS_SELECTOR, 'img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(10)
            return image_urls
            load_more_button = wd.find_element(By.CSS_SELECTOR, ".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [ ]:
image_urls = fetch_image_urls(query, count, wd)
print(image_urls)

<class 'list'>
[<selenium.webdriver.remote.webelement.WebElement (session="98a3f7071ee121ea70d70891bf23821e", element="76f4bf78-81d7-4a68-a3f0-b69c91bd0f10")>, <selenium.webdriver.remote.webelement.WebElement (session="98a3f7071ee121ea70d70891bf23821e", element="f370e482-8360-4515-b632-1a08e4b68d1d")>, <selenium.webdriver.remote.webelement.WebElement (session="98a3f7071ee121ea70d70891bf23821e", element="c13a2d64-4964-4dd5-a8c0-dfcf5ce25bc1")>, <selenium.webdriver.remote.webelement.WebElement (session="98a3f7071ee121ea70d70891bf23821e", element="a9aeae7c-6763-4b61-92c8-d2d455ed3c27")>, <selenium.webdriver.remote.webelement.WebElement (session="98a3f7071ee121ea70d70891bf23821e", element="c9108657-76ac-4d7d-87d3-bdc10a2de11f")>, <selenium.webdriver.remote.webelement.WebElement (session="98a3f7071ee121ea70d70891bf23821e", element="a2a45ca4-b92e-4f95-be08-12ffdc8beb0e")>, <selenium.webdriver.remote.webelement.WebElement (session="98a3f7071ee121ea70d70891bf23821e", element="9bfbb172-1483-486

Found: 175 image links, looking for more ...
{'https://www.natur-in-nrw.de/IMG/Tiere/Voegel/TV-245/makro.jpg', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSvbrDa0W5MYgjybWuKgRvLsZ7JAWnP_vCGzg&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcStqwFkB34Qq8ZbMGuDW-AAhqKIpOHYt3x1Qw&usqp=CAU', 'https://i.ytimg.com/vi/cbQuP4SkoiY/maxresdefault.jpg', 'https://www.lbv.de/files/user_upload/Bilder/Arten/Tiere/Vogel%20von%20A-Z/Singvogel/Star/Star_paar_mit-Brautgeschenk--Rosl-Roessner-lbv.jpg', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR9IrUOWTQc6y6S0_qq6qCU_lbgIEWzFuL5lg&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSI-JH7H5hfKzEZHE8CJjXwVF0-yX7EQhcuOg&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSWfOnGkbJj-5bX9K-lQ_XKqvXU07iG31yabw&usqp=CAU', 'https://www.vogelundnatur.de/wp-content/uploads/2014/02/star-vogel.jpg', 'https://www.nabu-shop.de/pub/media/catalog/product/cache/77956f6a106deb2c702539a11e471384/a/c/acd8ab

In [ ]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
        
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [ ]:
print(image_urls)

{'https://www.natur-in-nrw.de/IMG/Tiere/Voegel/TV-245/makro.jpg', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSvbrDa0W5MYgjybWuKgRvLsZ7JAWnP_vCGzg&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcStqwFkB34Qq8ZbMGuDW-AAhqKIpOHYt3x1Qw&usqp=CAU', 'https://i.ytimg.com/vi/cbQuP4SkoiY/maxresdefault.jpg', 'https://www.lbv.de/files/user_upload/Bilder/Arten/Tiere/Vogel%20von%20A-Z/Singvogel/Star/Star_paar_mit-Brautgeschenk--Rosl-Roessner-lbv.jpg', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR9IrUOWTQc6y6S0_qq6qCU_lbgIEWzFuL5lg&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSI-JH7H5hfKzEZHE8CJjXwVF0-yX7EQhcuOg&usqp=CAU', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSWfOnGkbJj-5bX9K-lQ_XKqvXU07iG31yabw&usqp=CAU', 'https://www.vogelundnatur.de/wp-content/uploads/2014/02/star-vogel.jpg', 'https://www.nabu-shop.de/pub/media/catalog/product/cache/77956f6a106deb2c702539a11e471384/a/c/acd8abc6-806d-4ee6-939d-13a1d0836157-1_2.jpg', 'htt

In [ ]:
for url in image_urls:
    path = f'{output_path}/{query}'
    if not os.path.exists(path):
        os.makedirs(path)
    persist_image(path, url)

SUCCESS - saved https://www.natur-in-nrw.de/IMG/Tiere/Voegel/TV-245/makro.jpg - as ./data/raw_google_images/star vogel\de04c6d5b6.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSvbrDa0W5MYgjybWuKgRvLsZ7JAWnP_vCGzg&usqp=CAU - as ./data/raw_google_images/star vogel\b94d1273b0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcStqwFkB34Qq8ZbMGuDW-AAhqKIpOHYt3x1Qw&usqp=CAU - as ./data/raw_google_images/star vogel\37e35ec6a1.jpg
SUCCESS - saved https://i.ytimg.com/vi/cbQuP4SkoiY/maxresdefault.jpg - as ./data/raw_google_images/star vogel\23c3a49916.jpg
SUCCESS - saved https://www.lbv.de/files/user_upload/Bilder/Arten/Tiere/Vogel%20von%20A-Z/Singvogel/Star/Star_paar_mit-Brautgeschenk--Rosl-Roessner-lbv.jpg - as ./data/raw_google_images/star vogel\ea0265751c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR9IrUOWTQc6y6S0_qq6qCU_lbgIEWzFuL5lg&usqp=CAU - as ./data/raw_google_images/star vogel\6cf6834fe3.jpg
SUCCESS - sav

SUCCESS - saved https://image.jimcdn.com/app/cms/image/transf/dimension=667x10000:format=jpg/path/sf1bcd3c03a164cc0/image/i12320b4d15bbd2d3/version/1525895061/image.jpg - as ./data/raw_google_images/star vogel\ed49d17f4f.jpg
SUCCESS - saved https://c8.alamy.com/compde/2g4nmht/der-gewohnliche-star-oder-europaische-star-sturnus-vulgaris-sammelt-wurmer-auf-einem-springend-rasen-vogel-mit-schnabel-voller-wurmer-nahaufnahme-eines-elternteils-der-sich-auf-der-nahrungssuche-befindet-2g4nmht.jpg - as ./data/raw_google_images/star vogel\4eade3c61e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQWprXyLb4-CXuj8_zM3EIcny8W0ukE4zjMZg&usqp=CAU - as ./data/raw_google_images/star vogel\3490e843e0.jpg
SUCCESS - saved https://www.naturgucker-magazin.de/typo3temp/pics/2b9d8e57d2.jpg - as ./data/raw_google_images/star vogel\0790f96399.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTvbCUXmxM3wAmknYy6vPZUQ243wtDNp7G4_Q&usqp=CAU - as ./data/raw_google_ima

SUCCESS - saved http://www.tiere-im-garten.de/steckbrief_star-Dateien/01.JPG - as ./data/raw_google_images/star vogel\a3eb402952.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSwtUOBzAbHicgflLgk_hiIRNOW0T7WU74olg&usqp=CAU - as ./data/raw_google_images/star vogel\ea73c9b2e8.jpg
SUCCESS - saved https://media.istockphoto.com/id/171255428/de/foto/europ%C3%A4ische-star-sturnus-vulgaris-die-auf-baumstumpf.jpg?s=612x612&w=0&k=20&c=929NnSGT0Zh_DCp8TolSJAUks9fBDGdUrKy2hz5D0hg= - as ./data/raw_google_images/star vogel\5050ee938c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSwTd7NjrRTvXp88y_f5yc1WutjLDK6f6gptQ&usqp=CAU - as ./data/raw_google_images/star vogel\34a6f27a28.jpg
SUCCESS - saved https://images.herz-fuer-tiere.de/images/8/0/1/1/91108-1-de-DE/Star_iStock_000012958810XSmall.jpg - as ./data/raw_google_images/star vogel\eaf52adfb2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQlI46yM3MpM5hKixeb48Eb-Woc2zTLua

SUCCESS - saved https://grundschulblogs.de/imgCache/15869.jpg - as ./data/raw_google_images/star vogel\2fd10713e4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQscCUPdlXCEM1nzPuDKdTJPoMHFpUKWzFjkg&usqp=CAU - as ./data/raw_google_images/star vogel\b048f1e12b.jpg
SUCCESS - saved https://www.vienna.at/2017/10/ABD0022-20171013-4-3-227036412731-3641x2731.jpg - as ./data/raw_google_images/star vogel\f24f136eb0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTe27cmQZH_Cn61XtlcO9QPK-C0dglr_PQGhA&usqp=CAU - as ./data/raw_google_images/star vogel\8425e09356.jpg
SUCCESS - saved https://image.jimcdn.com/app/cms/image/transf/none/path/sa0edae063b84f112/image/i60a9888d2751a00f/version/1563535292/star-sturnus-vulgaris.jpg - as ./data/raw_google_images/star vogel\fb50753be7.jpg
SUCCESS - saved https://image.jimcdn.com/app/cms/image/transf/dimension=513x10000:format=jpg/path/s443db976a885dc2c/image/i65c392dcd65a986b/version/1514275043/der-star-bild-